In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
train

In [ ]:
train['Survived'].value_counts(normalize=True)

In [ ]:
fig = plt.figure(figsize=(10,5))
sns.swarmplot(x='Pclass', y='Fare', data=train, hue='Survived')

In [ ]:
train.groupby(['Pclass', 'Sex'])['Survived'].value_counts(normalize=True)

In [ ]:
sns.histplot(train['Age'].dropna())

In [ ]:
plt.hist(train['Fare'], bins=10)
plt.xlabel('Fare')
plt.ylabel('Count')
plt.show()

In [ ]:
# объединим данные
full_data = [train, test]

for data in full_data:
    data.loc[data['Fare'] == 0, 'Fare'] = np.NaN

In [ ]:
sns.countplot(x='Embarked', data=train)

In [ ]:
sns.countplot(x='SibSp', data=train)

In [ ]:
sns.countplot(x='Parch', data=train)

In [ ]:
train

In [ ]:
# препроцессинг
for data in full_data:
    data['Title'] = data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [ ]:
train['Title'].value_counts()

In [ ]:
for data in full_data:
    data['Title'].replace(['Mme', 'Ms', 'Lady', 'Mlle', 'the Countess', 'Dona'], 'Miss', inplace=True)
    data['Title'].replace(['Major', 'Col', 'Capt', 'Don', 'Sir', 'Jonkheer'], 'Mr', inplace=True)
    
    data['Sex'].replace(['male', 'female'], [1,0], inplace=True)

In [ ]:
for data in full_data:
    data['Ticket_2letter'] = data.Ticket.apply(lambda x: x[:2])
    data['Ticket_len'] = data.Ticket.apply(lambda x: len(x))
    
    data['Cabin_num'] = data.Ticket.apply(lambda x: len(x.split()))
    data['Cabin_1letter'] = data.Ticket.apply(lambda x: x[:1])
    
    data['Fam_size'] = data['SibSp'] + data['Parch'] + 1

In [ ]:
train.Fam_size.unique()

In [ ]:
for data in full_data:
    data['Fam_type'] = pd.cut(data.Fam_size, [0,1,4,7,11], labels=['Solo', 'Small', 'Big', 'Very big'])

In [ ]:
# разработка модели

y = train['Survived']
features = ['Pclass', 'Fare', 'Title', 'Embarked', 'Fam_type', 'Ticket_len', 'Ticket_2letter']
X = train[features]

In [ ]:
numerical_cols = ['Fare']
categorical_cols = ['Pclass', 'Title', 'Embarked', 'Fam_type', 'Ticket_len', 'Ticket_2letter']
        

numerical_transformer = SimpleImputer(strategy='median')
       

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
      

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
       

titanic_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=0, n_estimators=500, max_depth=5))
])


#обучение модели
titanic_pipeline.fit(X,y)

In [ ]:
# применим модель на тестовых данных

X_test = test[features]
predictions = titanic_pipeline.predict(X_test)

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('titanic_predict.csv', index=False)